# Org Babel Setup

``` python
def assStr (name, literal=True):
  f = repr if literal else str
  return name + " = " + f(eval(name))
```

``` elisp
(message "%s" v)
```

``` elisp
(when (eq 'hline (cadr table))
  (setq table (cddr table)))
(cl-flet ((ellide-list (list length &optional (ellision "..."))
              (if (and length (length> list length))
                  (append (seq-subseq list 0 (/ length 2))
                          (list ellision)
                          (seq-subseq list (- (length list) (/ length 2))))
                list)))
    (mapcar (lambda (row) (if (listp row) (ellide-list row ncols) row))
            (ellide-list table nrows (make-list (length (car table)) "..."))))
```

``` elisp
(cl-mapcar #'list (number-sequence start (+ start (length list))) list)
```

``` elisp
(cl-loop with i = (1- start)
         for startp = t then nil
         for tail on table
         collect
         (cond ((atom (car tail)) (car tail))
               ((and startp (eq 'hline (cadr tail)))
                (cons index-name (car tail)))
               (t (cons (cl-incf i) (car tail)))))
```

# Setup

``` python
from typing import *
import pandas as pd
```

# Download Sessions

Set `data_dir` and ensure it exists.

``` python
from pathlib import Path
data_dir: Final[Path] = Path('./data/')
data_dir.mkdir(parents=True, exist_ok=True)
```

Set `manifest_path`.

``` python
manifest_path: Final[Path] = data_dir / 'manifest.json'
```

Create and initialize the project cache object

``` python
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
cache: Final = EcephysProjectCache.from_warehouse(manifest=manifest_path, timeout=30*60)
```

and obtain the sessions

``` python
sessions = cache.get_session_table()
```

Extract the session ids from `sessions` into `session_ids`

``` python
session_ids: Final[Sequence[int]] = list(sessions.index)
```

Extract a single session into `session`:

``` python
assert session_id in session_ids
session = cache.get_session_data(session_id) 
session.metadata
```

``` python
session.units.head()
```

``` python
session.stimulus_conditions.head()
```

``` python
session.stimulus_conditions['stimulus_name'].value_counts()
```

``` python
grouped_stimulus_presentations = session.stimulus_presentations.groupby('stimulus_name')
```

``` python
total_durations = grouped_stimulus_presentations['duration'].sum()
```

``` python
def presentation_type_spike_times(expr = None, mask = None, unit_names = session.units.index):
  presentations = session.stimulus_presentations
  if mask is not None:
    presentations = presentations[mask]
  if expr is not None:
    presentations = presentations.query(expr)
  return session.presentationwise_spike_times(presentations['stimulus_condition_id'], unit_names)

presentation_type_spike_times(expr="`stimulus_name` == 'drifting_gratings'").head(10)
```

``` python
spike_times_with_presentations_condition_names = \
  pd.merge(
    session.presentationwise_spike_times(
      session.stimulus_presentations['stimulus_condition_id'],
      session.units.index).reset_index(),
    session.stimulus_presentations['stimulus_name'],
    how='left',
    on='stimulus_presentation_id'
  )
```

``` python
def get_spike_times_df(unit_ids: Optional[Sequence[int]]=None):
  return pd.concat((
    (_df := pd.DataFrame(unit_spike_times, columns=['spike_time']),
     _df.insert(0,'unit_id',unit_id),
     _df)[-1]
    for (unit_id, unit_spike_times) in session.spike_times.items()
    if unit_ids is None or unit_id in unit_ids
  ), ignore_index=True, copy=False)

spike_times_df = get_spike_times_df()
```

``` python

```